In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer

In [2]:
train=pd.read_excel(r'C:\Users\SANIKA\Documents\Final_Train_DR_FEES.xlsx')

In [3]:
test=pd.read_excel(r'C:\Users\SANIKA\Documents\Final_Test_DR_FEES.xlsx')

In [4]:
train.shape,test.shape

((5961, 7), (1987, 6))

In [5]:
train.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees
0,"BHMS, MD - Homeopathy",24 years experience,100%,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100
1,"BAMS, MD - Ayurveda Medicine",12 years experience,98%,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350
2,"MBBS, MS - Otorhinolaryngology",9 years experience,NaN,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300
3,"BSc - Zoology, BAMS",12 years experience,NaN,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ₹250 Available on...",250
4,BAMS,20 years experience,100%,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5961 entries, 0 to 5960
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Qualification       5961 non-null   object
 1   Experience          5961 non-null   object
 2   Rating              2659 non-null   object
 3   Place               5936 non-null   object
 4   Profile             5961 non-null   object
 5   Miscellaneous_Info  3341 non-null   object
 6   Fees                5961 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 326.1+ KB


In [7]:
train.isnull().sum()

Qualification            0
Experience               0
Rating                3302
Place                   25
Profile                  0
Miscellaneous_Info    2620
Fees                     0
dtype: int64

In [8]:
train.nunique()

Qualification         1420
Experience              64
Rating                  51
Place                  877
Profile                  6
Miscellaneous_Info    2998
Fees                    40
dtype: int64

In [9]:
train.columns

Index(['Qualification', 'Experience', 'Rating', 'Place', 'Profile',
       'Miscellaneous_Info', 'Fees'],
      dtype='object')

In [10]:
train['Profile'].value_counts()

Dentist             1397
General Medicine    1173
Dermatologists      1034
Homeopath            843
Ayurveda             786
ENT Specialist       728
Name: Profile, dtype: int64

In [11]:
import sklearn.preprocessing as LabelEncoder

train.replace({'Dentist': 0,'General Medicine': 1,'Dermatologists': 2,'Homeopath': 3,'Ayurveda': 4,'ENT Specialist':5}, inplace = True)

In [12]:
train.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees
0,"BHMS, MD - Homeopathy",24 years experience,100%,"Kakkanad, Ernakulam",3,"100% 16 Feedback Kakkanad, Ernakulam",100
1,"BAMS, MD - Ayurveda Medicine",12 years experience,98%,"Whitefield, Bangalore",4,"98% 76 Feedback Whitefield, Bangalore",350
2,"MBBS, MS - Otorhinolaryngology",9 years experience,NaN,"Mathikere - BEL, Bangalore",5,NaN,300
3,"BSc - Zoology, BAMS",12 years experience,NaN,"Bannerghatta Road, Bangalore",4,"Bannerghatta Road, Bangalore ₹250 Available on...",250
4,BAMS,20 years experience,100%,"Keelkattalai, Chennai",4,"100% 4 Feedback Keelkattalai, Chennai",250


In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987 entries, 0 to 1986
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Qualification       1987 non-null   object
 1   Experience          1987 non-null   object
 2   Rating              897 non-null    object
 3   Place               1981 non-null   object
 4   Profile             1987 non-null   object
 5   Miscellaneous_Info  1153 non-null   object
dtypes: object(6)
memory usage: 93.3+ KB


In [14]:
test['Profile'].value_counts()

General Medicine    444
Dentist             441
Dermatologists      339
Homeopath           277
Ayurveda            251
ENT Specialist      235
Name: Profile, dtype: int64

In [15]:
import sklearn.preprocessing as LabelEncoder

test.replace({'Dentist': 0,'General Medicine': 1,'Dermatologists': 2,'Homeopath': 3,'Ayurveda': 4,'ENT Specialist':5}, inplace = True)

In [16]:
train['Experience'] = train['Experience'].str.replace('years experience','').astype(int)
test['Experience'] = test['Experience'].str.replace('years experience','').astype(int)

In [17]:
import re
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

train['Qualification'] = train['Qualification'].apply(clean_text)
test['Qualification'] = test['Qualification'].apply(clean_text)

def clean_place(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

train['Place'].fillna('missing', inplace=True)
test['Place'].fillna('missing', inplace=True)
train['Place'] = train['Place'].apply(clean_place)
test['Place'] = test['Place'].apply(clean_place)

In [18]:
train['Rating'].fillna('0%', inplace=True)
test['Rating'].fillna('0%', inplace=True)

def clean_rating(text):
    text = re.sub(r'%', '', str(text))
    return text

train['Rating'] = train['Rating'].apply(clean_rating)
train['Rating'] =train['Rating'].astype(int)

test['Rating'] = test['Rating'].apply(clean_rating)
test['Rating'] = test['Rating'].astype(int)

In [19]:
train['Miscellaneous_Info'].fillna('missing', inplace=True)
test['Miscellaneous_Info'].fillna('missing', inplace=True)

def get_feedback(feedback):
    feedback = re.findall(r'\d+ Feedback', str(feedback))
    if feedback == []:
        feedback = '0 Feedback'
        return feedback
    else:
        return feedback[0]

train['Feedback'] = train['Miscellaneous_Info'].apply(get_feedback)
train['Feedback'] = train['Feedback'].str.replace(' Feedback','')
train['Feedback'] = train['Feedback'].astype(int)

test['Feedback'] = test['Miscellaneous_Info'].apply(get_feedback)
test['Feedback'] = test['Feedback'].str.replace(' Feedback','')
test['Feedback'] = test['Feedback'].astype(int)

In [20]:
def get_fee(text):
    text = re.sub(r',', "", text)
    text = re.findall(r'₹\d+', text)
    if text != []:
        return text[0]
    else:
        return 0

train['Misc_Fees'] = train['Miscellaneous_Info'].apply(get_fee)
train['Misc_Fees'] = train['Misc_Fees'].str.replace('₹','')
train['Misc_Fees'].fillna(50, inplace=True)
train['Misc_Fees'] = train['Misc_Fees'].astype(int)

test['Misc_Fees'] = test['Miscellaneous_Info'].apply(get_fee)
test['Misc_Fees'] = test['Misc_Fees'].str.replace('₹','')
test['Misc_Fees'].fillna(50, inplace=True)
test['Misc_Fees'] = test['Misc_Fees'].astype(float)

In [21]:
def clean_misc(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

train['Miscellaneous_Info'] = train['Miscellaneous_Info'].apply(clean_misc)
test['Miscellaneous_Info'] = test['Miscellaneous_Info'].apply(clean_misc)

In [22]:
x = train.drop(labels=['Fees'], axis=1)
y = train['Fees'].values

In [23]:
# Encoding
enc = OrdinalEncoder()
x= enc.fit_transform(x)

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn import linear_model
def maxr2_score(regr,x,y):
    max_r_score=0
    for r_state in range(42,100):
        x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=r_state,test_size=0.20)
        regr.fit(x_train,y_train)
        y_pred=regr.predict(x_test)
        r2_scr=r2_score(y_test,y_pred)
        print('r2_score corresponding to random state:',r_state,'is',r2_scr)
        if r2_scr>max_r_score:
            max_r_score=r2_scr
            final_r_state=r_state

    print('max r2 score corresponding to',final_r_state,'is:',max_r_score)
    return final_r_state

In [57]:
#lets use Linear Regression and check max_r2_score
from sklearn.linear_model import LinearRegression
lreg=LinearRegression()
r_state=maxr2_score(lreg,x,y)

r2_score corresponding to random state: 42 is 0.13274491749603423
r2_score corresponding to random state: 43 is 0.13090034799249373
r2_score corresponding to random state: 44 is 0.138572561145749
r2_score corresponding to random state: 45 is 0.11451591462107114
r2_score corresponding to random state: 46 is 0.10800146606829442
r2_score corresponding to random state: 47 is 0.10007879448836854
r2_score corresponding to random state: 48 is 0.10805001281039273
r2_score corresponding to random state: 49 is 0.1157781408102937
r2_score corresponding to random state: 50 is 0.10691045256874299
r2_score corresponding to random state: 51 is 0.12003386200484156
r2_score corresponding to random state: 52 is 0.06606677876546563
r2_score corresponding to random state: 53 is 0.09421915582199847
r2_score corresponding to random state: 54 is 0.12148874170004553
r2_score corresponding to random state: 55 is 0.11722394373274803
r2_score corresponding to random state: 56 is 0.12947769540078535
r2_score corr

In [58]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
neighbors={'n_neighbors':range(1,30)}
knr=KNeighborsRegressor()
gknr=GridSearchCV(knr,neighbors,cv=10)
gknr.fit(x,y)
gknr.best_params_

{'n_neighbors': 29}

In [59]:
#lets use KNN regression and check max r2 score
knr=KNeighborsRegressor(n_neighbors=29)
r_state=maxr2_score(knr,x,y)

r2_score corresponding to random state: 42 is 0.13823909598054307
r2_score corresponding to random state: 43 is 0.129466354458745
r2_score corresponding to random state: 44 is 0.16049937623435773
r2_score corresponding to random state: 45 is 0.12711081717631956
r2_score corresponding to random state: 46 is 0.11566346139438155
r2_score corresponding to random state: 47 is 0.09896896966526703
r2_score corresponding to random state: 48 is 0.11823072253314237
r2_score corresponding to random state: 49 is 0.13282123247423194
r2_score corresponding to random state: 50 is 0.12846487178561738
r2_score corresponding to random state: 51 is 0.12081268476693707
r2_score corresponding to random state: 52 is 0.09984945802553524
r2_score corresponding to random state: 53 is 0.10375553580200758
r2_score corresponding to random state: 54 is 0.14337067970965622
r2_score corresponding to random state: 55 is 0.11276327578510403
r2_score corresponding to random state: 56 is 0.14085524856301712
r2_score cor

In [60]:
from sklearn.model_selection import cross_val_score
print('Mean r2 score for Linear Regression:',cross_val_score(lreg,x,y,cv=5,scoring='r2').mean())
print('standard deviation in r2 score for Linear Regression: ',cross_val_score(lreg,x,y,cv=5,scoring='r2').std())
print()
print('Mean r2 score for KNN Regression:',cross_val_score(knr,x,y,cv=5,scoring='r2').mean())
print('standard deviation in r2 score for KNeighborsRegressor: ',cross_val_score(knr,x,y,cv=5,scoring='r2').std())
print()

Mean r2 score for Linear Regression: 0.11107771474067099
standard deviation in r2 score for Linear Regression:  0.010066988997402134

Mean r2 score for KNN Regression: 0.11976187361027887
standard deviation in r2 score for KNeighborsRegressor:  0.011354001849983772



In [61]:
#lets check lasso regression & find alpha
from sklearn.linear_model import Lasso
lsreg=Lasso()
parameters={'alpha':[0.001,0.01,0.1,1]}
clf=GridSearchCV(lsreg,parameters,cv=10)
clf.fit(x,y)
clf.best_params_

{'alpha': 0.1}

In [62]:
#lets check max_r2_score with alpha=0.1
lsreg=Lasso(alpha=0.1)
r_state=maxr2_score(lsreg,x,y)

r2_score corresponding to random state: 42 is 0.13268165802339293
r2_score corresponding to random state: 43 is 0.13088504949509794
r2_score corresponding to random state: 44 is 0.13857126963337818
r2_score corresponding to random state: 45 is 0.11449202201597775
r2_score corresponding to random state: 46 is 0.10801955666189178
r2_score corresponding to random state: 47 is 0.10005188144746413
r2_score corresponding to random state: 48 is 0.10804828698478419
r2_score corresponding to random state: 49 is 0.11578377487037639
r2_score corresponding to random state: 50 is 0.10689141906091426
r2_score corresponding to random state: 51 is 0.12000260849253142
r2_score corresponding to random state: 52 is 0.06616562065741005
r2_score corresponding to random state: 53 is 0.09424022214772465
r2_score corresponding to random state: 54 is 0.12143063221435546
r2_score corresponding to random state: 55 is 0.11719377689053723
r2_score corresponding to random state: 56 is 0.12944759995476307
r2_score c

In [63]:
print('Mean r2 score for Lasso Regression:',cross_val_score(lsreg,x,y,cv=5,scoring='r2').mean())
print('standard deviation in r2 score for Lasso Regression: ',cross_val_score(lsreg,x,y,cv=5,scoring='r2').std())
print()

Mean r2 score for Lasso Regression: 0.11107908812808012
standard deviation in r2 score for Lasso Regression:  0.010033285776466784



In [64]:
#Now we will try to boost 
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')
gbr=GradientBoostingRegressor()
parameters={'learning_rate':[0.001,0.01,0.1,1],'n_estimators':[10,100,500,1000]}
clf=GridSearchCV(gbr,parameters,cv=5)
clf.fit(x,y)
clf.best_params_

{'learning_rate': 0.1, 'n_estimators': 500}

In [65]:
gbr=GradientBoostingRegressor(learning_rate=0.1 ,n_estimators=500 )
print('Mean r2 score for GradientBoostingRegressor:',cross_val_score(gbr,x,y,cv=5,scoring='r2').mean())
print('standard deviation in r2 score for GradientBoostingRegressor: ',cross_val_score(gbr,x,y,cv=5,scoring='r2').std())
print()

Mean r2 score for GradientBoostingRegressor: 0.21024925867578315
standard deviation in r2 score for GradientBoostingRegressor:  0.03079406612859646



In [66]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')
ada_reg=AdaBoostRegressor()
parameters={'learning_rate':[0.001,0.01,0.1,1],'n_estimators':[10,100,500,1000],
            'base_estimator':[lreg,lsreg,DecisionTreeRegressor()]}
clf=GridSearchCV(ada_reg,parameters,cv=5)
clf.fit(x,y)
clf.best_params_

{'base_estimator': DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best'),
 'learning_rate': 1,
 'n_estimators': 1000}

In [69]:
ada_reg=AdaBoostRegressor(learning_rate=1 ,n_estimators=1000 )
print('Mean r2 score for AdaBoostRegressor:',cross_val_score(ada_reg,x,y,cv=5,scoring='r2').mean())
print('standard deviation in r2 score for AdaBoostRegressor: ',cross_val_score(ada_reg,x,y,cv=5,scoring='r2').std())
print()

Mean r2 score for AdaBoostRegressor: 0.08683536197823964
standard deviation in r2 score for AdaBoostRegressor:  0.007758425849826657



In [68]:
r_state=maxr2_score(ada_reg,x,y)

r2_score corresponding to random state: 42 is 0.11511739483869465
r2_score corresponding to random state: 43 is 0.0965403542082588
r2_score corresponding to random state: 44 is 0.1133549971998904
r2_score corresponding to random state: 45 is 0.08920266647873465
r2_score corresponding to random state: 46 is 0.06168824853970456
r2_score corresponding to random state: 47 is 0.0695869140831703
r2_score corresponding to random state: 48 is 0.09731315972507792
r2_score corresponding to random state: 49 is 0.07611545597707792
r2_score corresponding to random state: 50 is 0.1003646744990141
r2_score corresponding to random state: 51 is 0.10645626965265065
r2_score corresponding to random state: 52 is 0.08416223469571404
r2_score corresponding to random state: 53 is 0.054092912230573
r2_score corresponding to random state: 54 is 0.11130790521851586
r2_score corresponding to random state: 55 is 0.07873195489682094
r2_score corresponding to random state: 56 is 0.059646065189426056
r2_score corres

In [70]:
r_state=maxr2_score(gbr,x,y)

r2_score corresponding to random state: 42 is 0.22740353619329756
r2_score corresponding to random state: 43 is 0.19626072633456404
r2_score corresponding to random state: 44 is 0.2447827342429577
r2_score corresponding to random state: 45 is 0.1907866466050402
r2_score corresponding to random state: 46 is 0.17765676675645914
r2_score corresponding to random state: 47 is 0.1980149401584126
r2_score corresponding to random state: 48 is 0.1850051798717287
r2_score corresponding to random state: 49 is 0.20471080554064047
r2_score corresponding to random state: 50 is 0.16836427185011493
r2_score corresponding to random state: 51 is 0.2051298351077152
r2_score corresponding to random state: 52 is 0.169710761564296
r2_score corresponding to random state: 53 is 0.17843921305617383
r2_score corresponding to random state: 54 is 0.24580348775731975
r2_score corresponding to random state: 55 is 0.21880664579785591
r2_score corresponding to random state: 56 is 0.22184801167141466
r2_score correspo

In [71]:
#We tried all the models and till now the best is
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state =63 ,test_size = 0.20)
gbr=GradientBoostingRegressor()
gbr.fit(x_train, y_train)
y_pred=gbr.predict(x_test)

In [72]:
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('r2 score is:',r2_score(y_test, y_pred))

RMSE: 166.68128110624946
r2 score is: 0.2480469655687395


I choose GradientBoostingRegressor as my final model because it performed very well on dataset

In [73]:
from sklearn.externals import joblib
#save the model 
joblib.dump(gbr, 'Project13.obj')

['Project13.obj']